## BPE 실습

In [ ]:
import re
import collections
from IPython.display import display, Markdown, Latex

bpe 실행 횟수 지정

In [ ]:
num_merges = 10

BPE 입력 생성. 이때 </w>는 단어의 맨끝에 붙여서 단어의 끝을 나타낸다.

In [ ]:
dictionary ={
    'l o w </w>' : 5,
    'l o w e r </w>' : 2,
    'n e a r e s t </w>' : 6,
    'w i d e s t </w>' : 3
}

위 dictionary에서 low는 5번, lower는 2번, nearest는 6번, widest는 3번 나타났다는 것을 의미한다.

BPE 코드

In [ ]:
def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

# pairs = {('l', 'o') : 2, ('o', 'w') : 2, ...} 이런 방식으로 pair가 매번 만들어진다.

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))  # 가장 많이 나타난 쌍에 대해 특수문자들을 escape 처리한다. ex) ('e', 's') => e\ s
    print(bigram)
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        print(w_out)
        v_out[w_out] = v_in[word]
    return v_out

bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)  # 쌍들을 만든다
    best = max(pairs, key=pairs.get)  # 가장 많이 나타난 쌍을 저장한다
    dictionary = merge_dictionary(best, dictionary) # 가장 많이 나타난 쌍을 단어마다 붙여놓는다. ex) l o w e s t => l o w es t

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

### Iteration 1

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'a'): 6, ('a', 'r'): 6, ('r', 'e'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
e\ s
l o w </w>
l o w e r </w>
n e a r es t </w>
w i d es t </w>
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e a r es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'a'): 6, ('a', 'r'): 6, ('r', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
es\ t
l o w </w>
l o w e r </w>
n e a r est </w>
w i d est </w>
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e a r est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('nea', 'r'): 6, ('r', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
nea\ r
low </w>
low e r </w>
near est</w>
w i d est</w>
new merge: ('nea', 'r')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'near est</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('near', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
near\ est</w>
low </w>
low e r </w>
nearest</w>
w i d est</w>
new merge: ('near', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'nearest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
low\ </w>
low</w>
low e r </w>
nearest</w>
w i d est</w>
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'nearest</w>': 6, 'w i d est</w>': 3}


In [ ]:
print(bpe_codes)

{('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'a'): 6, ('nea', 'r'): 7, ('near', 'est</w>'): 8, ('low', '</w>'): 9}


In [ ]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))

    pairs = get_pairs(word)    

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))

        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf'))) # 두 덩어리씩 묶어서 그 덩어리가 bpe code게 있는지 확인한다.
        print("candidate for merging: {}".format(bigram))
        if bigram not in bpe_codes: # 없다면 subword merge 단계를 종료한다
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # 특별 토큰인 </w>는 출력하지 않는다.
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)

    return word

In [ ]:
encode("loki")

__word split into characters:__ <tt>('l', 'o', 'k', 'i', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('k', 'i'), ('o', 'k'), ('l', 'o'), ('i', '</w>')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'k', 'i', '</w>')


__Iteration 2:__

bigrams in the word: {('k', 'i'), ('i', '</w>'), ('lo', 'k')}
candidate for merging: ('k', 'i')


__Candidate not in BPE merges, algorithm stops.__

('lo', 'k', 'i')

In [ ]:
encode("lowest")

__word split into characters:__ <tt>('l', 'o', 'w', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('l', 'o'), ('s', 't'), ('t', '</w>'), ('e', 's'), ('w', 'e'), ('o', 'w')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '</w>')


__Iteration 2:__

bigrams in the word: {('l', 'o'), ('t', '</w>'), ('es', 't'), ('w', 'es'), ('o', 'w')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '</w>')


...

bigrams in the word: {('lo', 'w'), ('w', 'est</w>')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'est</w>')


__Iteration 6:__

bigrams in the word: {('low', 'est</w>')}
candidate for merging: ('low', 'est</w>')


__Candidate not in BPE merges, algorithm stops.__

('low', 'est')

In [ ]:
encode("lowing")

__word split into characters:__ <tt>('l', 'o', 'w', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('l', 'o'), ('i', 'n'), ('g', '</w>'), ('w', 'i'), ('n', 'g'), ('o', 'w')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'i', 'n', 'g', '</w>')


__Iteration 2:__

bigrams in the word: {('lo', 'w'), ('i', 'n'), ('g', '</w>'), ('w', 'i'), ('n', 'g')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'i', 'n', 'g', '</w>')


__Iteration 3:__

bigrams in the word: {('n', 'g'), ('g', '</w>'), ('low', 'i'), ('i', 'n')}
candidate for merging: ('n', 'g')


__Candidate not in BPE merges, algorithm stops.__

('low', 'i', 'n', 'g')

In [ ]:
encode("highing")

__word split into characters:__ <tt>('h', 'i', 'g', 'h', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('h', 'i'), ('i', 'n'), ('g', '</w>'), ('n', 'g'), ('g', 'h'), ('i', 'g')}
candidate for merging: ('h', 'i')


__Candidate not in BPE merges, algorithm stops.__

('h', 'i', 'g', 'h', 'i', 'n', 'g')

subword의 어느 부분도 나타나지 않을 경우 글자 하나하나가 분리되어서 구별된다.

In [ ]:
encode("highest")

__word split into characters:__ <tt>('h', 'i', 'g', 'h', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('h', 'i'), ('s', 't'), ('t', '</w>'), ('h', 'e'), ('e', 's'), ('g', 'h'), ('i', 'g')}
candidate for merging: ('e', 's')
word after merging: ('h', 'i', 'g', 'h', 'es', 't', '</w>')


__Iteration 2:__

bigrams in the word: {('h', 'i'), ('t', '</w>'), ('h', 'es'), ('es', 't'), ('g', 'h'), ('i', 'g')}
candidate for merging: ('es', 't')
word after merging: ('h', 'i', 'g', 'h', 'est', '</w>')


...

bigrams in the word: {('h', 'i'), ('est', '</w>'), ('h', 'est'), ('g', 'h'), ('i', 'g')}
candidate for merging: ('est', '</w>')
word after merging: ('h', 'i', 'g', 'h', 'est</w>')


__Iteration 4:__

bigrams in the word: {('g', 'h'), ('h', 'i'), ('h', 'est</w>'), ('i', 'g')}
candidate for merging: ('g', 'h')


__Candidate not in BPE merges, algorithm stops.__

('h', 'i', 'g', 'h', 'est')

## Sentence Piece

구글에서 만든 패키지로 사전 tokenize 작업 없이 단어를 분리해준다.

In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00


In [ ]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f65e24f2e60>)

## IMDB Review Subword Tokenize 실습

In [ ]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df.review

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [ ]:
len(train_df)

50000

IMDB Review에 현재 50000개의 review가 있다.

IMDB에 대해 조사하기

In [ ]:
train_df.isna().values.any()

False

결측치는 없다.

SentencePiece를 사용해보기 전에 원래 하던데로 tokenize 등의 작업을 해보기로 한다.

우선 각 텍스트에서 문자와 숫자 이외의 모든 캐릭터를 제거한다. 그리고 전부 소문자로 바꿔준다.

In [ ]:
import re

def clean_sentence(sentence):
    return re.sub(r"[^a-z0-9]+", " ", sentence.lower())

In [ ]:
train_df["cleaned_review"] = train_df.review.apply(clean_sentence)

In [ ]:
train_df.cleaned_review.head()

0    my family and i normally do not watch local mo...
1    believe it or not this was at one time the wor...
2    after some internet surfing i found the homefr...
3    one of the most unheralded great works of anim...
4    it was the sixties and anyone with long hair a...
Name: cleaned_review, dtype: object

In [ ]:
from nltk.tokenize import word_tokenize
import nltk

nltk.download("punkt")

result = [word_tokenize(sentence) for sentence in train_df.cleaned_review]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


단어들의 빈도수 확인

In [ ]:
from collections import defaultdict

word_count = defaultdict(int)

for tokenlist in result:
    for token in tokenlist:
        word_count[token] += 1

In [ ]:
word_count = pd.Series(word_count)

In [ ]:
word_count.shape

(101244,)

In [ ]:
word_count[word_count == word_count.min()]

diether            1
ocampo             1
brekinridge        1
flopperoo          1
soupcon            1
                  ..
tamping            1
langoria           1
splainin           1
apocalyptically    1
eotw               1
Length: 38289, dtype: int64

In [ ]:
word_count[word_count == word_count.min()].count()

38289

25% 정도의 단어가 1번 밖에 나타나지 않는다.

In [ ]:
word_count[word_count == word_count.max()]

the    668009
dtype: int64

In [ ]:
word_count.mean()

118.3603472798388

In [ ]:
word_count[word_count != 1].sort_values()

deware            2
suntanned         2
klimovsky         2
nbb               2
donee             2
              ...  
to           268125
of           289414
a            323036
and          324443
the          668009
Length: 62955, dtype: int64

In [ ]:
word_count[word_count == 2].count()

12062

두 번만 나타나는 단어들의 비율도 전체의 10%이다.

In [ ]:
word_count[word_count == 3].count()

6666

In [ ]:
word_count[word_count == 4].count()

4485

In [ ]:
38289 + 12062 + 6666 + 4485

61502

만약 최소 word count를 5로 잡으면 101244개 중 38289 + 12062 + 6666 + 4485 = 61502개의 단어가 버려지는 셈이다.

In [ ]:
word_count[word_count == 5].count()

3293

In [ ]:
word_count[word_count == 6].count()

2624

In [ ]:
word_count[word_count == 7].count()

2195

In [ ]:
word_count[word_count == 8].count()

1814

FastText사용

min count = 5 로 설정해서 실행해보기

In [ ]:
from gensim.models import FastText

model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
model.wv.most_similar("videogame")

[('videogames', 0.9683234691619873),
 ('video', 0.8710603713989258),
 ('videodrome', 0.8410304188728333),
 ('videostore', 0.8183623552322388),
 ('hdtv', 0.8065782785415649),
 ('videotape', 0.8056202530860901),
 ('videotapes', 0.8027333617210388),
 ('videos', 0.8023756146430969),
 ('videocassette', 0.7905303835868835),
 ('ps2', 0.7726799845695496)]

In [ ]:
model.wv.most_similar("protractor")

[('tractor', 0.7792906165122986),
 ('yyz', 0.7300223708152771),
 ('protracted', 0.7165572643280029),
 ('sycophantic', 0.7110646963119507),
 ('adrenalin', 0.7106966376304626),
 ('benefactor', 0.7085446119308472),
 ('proto', 0.7081419229507446),
 ('protocol', 0.707724392414093),
 ('fertilizer', 0.7052549123764038),
 ('zippo', 0.7006384134292603)]

이번에는 빈도수를 3개로 바꿔서 시도해보았다.

In [ ]:
model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
model.wv.most_similar("video art")

[('video', 0.9334495067596436),
 ('videostore', 0.9133040308952332),
 ('videogames', 0.8944391012191772),
 ('videos', 0.8667849898338318),
 ('videotape', 0.8629374504089355),
 ('videotapes', 0.8596477508544922),
 ('videodrome', 0.8526623249053955),
 ('videocassette', 0.8510347604751587),
 ('hdtv', 0.8163554668426514),
 ('videotaped', 0.809007465839386)]

In [ ]:
model.wv.most_similar("pharaoh")

[('pharaohs', 0.9667155742645264),
 ('pharoah', 0.8169096112251282),
 ('kriemhild', 0.7750786542892456),
 ('tiberius', 0.773655891418457),
 ('nebula', 0.7655394673347473),
 ('abdalla', 0.7607023119926453),
 ('fdr', 0.7594434022903442),
 ('eglantine', 0.7587003111839294),
 ('xv', 0.7574669718742371),
 ('samoilova', 0.7536780834197998)]

In [ ]:
model.wv.most_similar("mesmerize")

[('mesmerized', 0.8457523584365845),
 ('mesmerizing', 0.8360087871551514),
 ('mesmerised', 0.8061157464981079),
 ('mesmerising', 0.7855075597763062),
 ('astound', 0.7343807816505432),
 ('captivate', 0.7065714001655579),
 ('enthralling', 0.700756311416626),
 ('captivates', 0.7000594139099121),
 ('enthrall', 0.6999481916427612),
 ('astounding', 0.6949604153633118)]

In [ ]:
model.wv.most_similar("kamehameha")

[('mameha', 0.8253352046012878),
 ('markham', 0.8194396495819092),
 ('neha', 0.8023437261581421),
 ('katzman', 0.7988134026527405),
 ('seitzman', 0.7917552590370178),
 ('katzir', 0.7885951995849609),
 ('ezra', 0.7802041172981262),
 ('jorja', 0.7797974348068237),
 ('szifron', 0.7729490995407104),
 ('jiah', 0.7723050117492676)]

In [ ]:
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df.review))

Sentence Piece로 각 단어 집합과 각 단어에 고유한 정수를 부여한다.

- input: 학습시킬 파일
- model_prefix: 모델 이름
- vocab_size: 단어 집합의 크기
- model_type: 사용할 모델 (unigram(default), bpe, char, word)
- max_sentence_length: 문장 최대 길이

In [ ]:
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=50000 --model_type=bpe --max_sentence_length=9999')

In [ ]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
12675,anky,-12672
20442,▁engross,-20439
16398,▁Bax,-16395
31987,▁fanciful,-31984
32793,▁presenter,-32790
8652,eland,-8649
42558,pie,-42555
6711,▁Devil,-6708
11672,▁hypoc,-11669
19643,▁prehistoric,-19640


In [ ]:
vocab_list.shape

(50000, 2)

In [ ]:
vocab_list.iloc[:, 0].value_counts().sort_values()

<unk>    1
or       1
es       1
▁o       1
</s>     1
        ..
3        1
5        1
8        1
▁        1
6        1
Name: 0, Length: 49998, dtype: int64

모든 토큰들이 unique하게 만들어진 것을 볼 수 있다.

word to integer sequence 작업

In [ ]:
sp = spm.SentencePieceProcessor()
vocab_file = 'imdb.model'
sp.load(vocab_file)

True

In [ ]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))  # subword sequence
  print(sp.encode_as_ids(line))     # integer sequence
  print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 624, 49950, 49926, 139, 170, 378, 30, 58, 73, 413, 49945]

I have waited a long time for someone to film
['▁I', '▁have', '▁waited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 9800, 4, 668, 285, 93, 1079, 33, 91]



In [ ]:
sp.GetPieceSize()

50000

In [ ]:
sp.IdToPiece(430)

'▁character'

In [ ]:
sp.PieceToId('▁character')

430

In [ ]:
sp.DecodeIds([41, 142, 9800, 4, 668, 285, 93, 1079, 33, 91])

'I have waited a long time for someone to film'

In [ ]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁waited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 142, 9800, 4, 668, 285, 93, 1079, 33, 91]


## 네이버 영화 리뷰 토큰화 (한글)

In [2]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f4573519390>)

In [5]:
naver_df = pd.read_table('ratings.txt')
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [6]:
naver_df.shape

(200000, 3)

In [7]:
naver_df.isna().values.any()

True

In [8]:
naver_df.dropna(how = 'any', inplace=True)
naver_df.isna().values.any()

False

In [9]:
naver_df.shape

(199992, 3)

원래 하던 tokenize 및 word embedding 방법

In [10]:
naver_df.columns

Index(['id', 'document', 'label'], dtype='object')

한글 이외의 다른 문자들을 전부 제거

In [12]:
naver_df.document = naver_df.document.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "")
naver_df.head()

<ipython-input-12-1e48ef8d20a2>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  naver_df.document = naver_df.document.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "")


,id,document,label
0,8112052,어릴때보고지금다시봐도재밌어요ㅋㅋ,1
1,8132799,디자인을배우는학생으로외국디자이너와그들이일군전통을통해발전해가는문화산업이부러웠는데사실우...,1
2,4655635,폴리스스토리시리즈는부터뉴까지버릴께하나도없음최고,1
3,9251303,와연기가진짜개쩔구나지루할거라고생각했는데몰입해서봤다그래이런게진짜영화지,1
4,10067386,안개자욱한밤하늘에떠있는초승달같은영화,1


In [14]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.3 MB/s eta 0:00:00


Okt를 활용한 한글 형태소 분석 (오래걸린다)

불용어는 빼준다

In [15]:
from konlpy.tag import Okt
from tqdm import tqdm

stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

okt = Okt()

tokenized_data = []
for sentence in tqdm(naver_df.document):
    tokenized_sentence = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    tokenized_data.append(stopwords_removed_sentence)

100%|██████████| 199992/199992 [1:12:22<00:00, 46.05it/s]


In [16]:
max(len(review) for review in tokenized_data)

68

In [17]:
tokenized_data[0]

['어리다', '때', '보고', '지금', '다시', '보다', '재밌다', 'ㅋㅋ']

각 token의 빈도수 체크

In [19]:
from collections import defaultdict

word_count = defaultdict(int)

for review in tokenized_data:
    for token in review:
        word_count[token] += 1

In [23]:
word_count = pd.Series(word_count).sort_values(ascending=False)

In [26]:
word_count.shape

(52703,)

In [24]:
word_count.head()

영화    59842
보다    50443
을     29986
없다    20434
이다    18275
dtype: int64

In [25]:
word_count[word_count == 1].count()

22066

절반이 1번만 나타난 단어들이다.

In [27]:
word_count[word_count == 2].count()

6626

In [28]:
word_count[word_count == 3].count()

3601

In [29]:
word_count[word_count == 4].count()

2463

In [30]:
word_count[word_count == 5].count()

1905

빈도수가 4 이상인 단어들만 embedding을 했다. 이렇게 해도 절반 이상의 단어가 날아간 거다.

In [31]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, vector_size=100, window=5, min_count=4, workers=4, sg=1)

In [32]:
model.wv.vectors.shape

(20410, 100)

In [33]:
print(model.wv.most_similar("최민식"))

[('김명민', 0.8212376236915588), ('김창완', 0.816953182220459), ('채민서', 0.8151782155036926), ('한석규', 0.8126875162124634), ('조재현', 0.8109741806983948), ('이정현', 0.8091638684272766), ('문소리', 0.8079192042350769), ('이정재', 0.8066917657852173), ('설경구', 0.8036688566207886), ('안성기', 0.8010847568511963)]


In [35]:
print(model.wv.most_similar("ㅋㅋ"))

[('ㅋㅋㅋ', 0.9446472525596619), ('ㅋㅋㅋㅋ', 0.8899937272071838), ('ㅋ', 0.869400143623352), ('ㅋㅋㅋㅋㅋ', 0.8309749364852905), ('ㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 0.8304468989372253), ('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 0.8237006664276123), ('ㅋㅋㅋㅋㅋㅋ', 0.8108184933662415), ('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 0.8077812790870667), ('ㅋㅋㅋㅋㅋㅋㅋㅋ', 0.7952751517295837), ('ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', 0.78778076171875)]


In [37]:
print(model.wv.most_similar("싫다"))

[('싫어지다', 0.7149919867515564), ('거북하다', 0.7129932641983032), ('꼴', 0.6819781064987183), ('듣기', 0.6716106534004211), ('역겹다', 0.6528339385986328), ('시름', 0.639968752861023), ('찍히다', 0.6175423860549927), ('모자라', 0.6172111630439758), ('꼴다', 0.6139127016067505), ('시르다', 0.6101692914962769)]


In [41]:
print(model.wv.most_similar("쩐다"))

[('쩌러', 0.8121099472045898), ('쩔다', 0.8051066994667053), ('우와', 0.7593398094177246), ('쩔어요', 0.7587863802909851), ('존잼', 0.7502408623695374), ('ㄷㄷㄷ', 0.7500672936439514), ('ㅎㄷㄷ', 0.7456656098365784), ('작렬', 0.7446728944778442), ('죽여주다', 0.7408681511878967), ('졸잼', 0.7380020618438721)]


텍스트 파일 저장

In [42]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df.document))

한글 토큰화 작업 중

In [43]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [44]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list.head(10)

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,영화,0
4,ᄏᄏ,-1
5,니다,-2
6,는데,-3
7,너무,-4
8,재미,-5
9,하고,-6


In [45]:
sp = spm.SentencePieceProcessor()
vocab_file = 'naver.model'
sp.load(vocab_file)

True

In [46]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
  print(line)
  print(sp.encode_as_pieces(line))
  print(sp.encode_as_ids(line))
  print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁', '것도', '▁영화', '냐', '.']
[510, 2207, 3383, 181, 162, 3584, 0]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화', '입니다', '▁ᄏᄏ']
[108, 1014, 162, 97, 1487]



In [47]:
sp.GetPieceSize()

5000

In [48]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화', '입니다', '▁ᄏᄏ']
[108, 1014, 162, 97, 1487]


SentencePiece 패키지를 활용하면 언어에 상관없이 단어들을 subword 단위로 분리해낸다.

## SubwordTextEncoder

Tensorflow에서 사용할 수 있는 subword Tokenizer로 BPE의 변형인 WordPiece Model을 사용함. 

IMDB 실습 Again

In [49]:
!pip install tensorflow_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds

In [51]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f452d853df0>)

In [52]:
train_df = pd.read_csv('IMDb_Reviews.csv')

In [53]:
train_df.review.head()

0    My family and I normally do not watch local mo...
1    Believe it or not, this was at one time the wo...
2    After some internet surfing, I found the "Home...
3    One of the most unheralded great works of anim...
4    It was the Sixties, and anyone with long hair ...
Name: review, dtype: object

Subword로 이루어진 단어 집합 생성 및 고유한 정수값 부여

deprecated라고 되어 있는데 아직까지는 뚜렷한 대체제가 없다고 한다. tf text에서 구현되고 있다는 얘기는 있다.

In [54]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(train_df.review, target_vocab_size=2**13)

In [55]:
print(tokenizer.subwords[:100])

['the_', ', ', '. ', 'a_', 'and_', 'of_', 'to_', 's_', 'is_', 'br', 'in_', 'I_', 'that_', 'this_', 'it_', ' /><', ' />', 'was_', 'The_', 't_', 'as_', 'with_', 'for_', '.<', 'on_', 'but_', 'movie_', 'are_', ' (', 'have_', 'his_', 'film_', 'not_', 'be_', 'you_', 'ing_', ' "', 'ed_', 'it', 'd_', 'an_', 'at_', 'by_', 'he_', 'one_', 'who_', 'from_', 'y_', 'or_', 'e_', 'like_', 'all_', '" ', 'they_', 'so_', 'just_', 'has_', ') ', 'about_', 'her_', 'out_', 'This_', 'some_', 'movie', 'ly_', 'film', 'very_', 'more_', 'It_', 'what_', 'would_', 'when_', 'if_', 'good_', 'up_', 'which_', 'their_', 'only_', 'even_', 'my_', 'really_', 'had_', 'can_', 'no_', 'were_', 'see_', '? ', 'she_', 'than_', '! ', 'there_', 'been_', 'get_', 'into_', 'will_', ' - ', 'much_', 'n_', 'because_', 'ing']


In [56]:
print(train_df['review'][20])

Pretty bad PRC cheapie which I rarely bother to watch over again, and it's no wonder -- it's slow and creaky and dull as a butter knife. Mad doctor George Zucco is at it again, turning a dimwitted farmhand in overalls (Glenn Strange) into a wolf-man. Unfortunately, the makeup is virtually non-existent, consisting only of a beard and dimestore fangs for the most part. If it were not for Zucco and Strange's presence, along with the cute Anne Nagel, this would be completely unwatchable. Strange, who would go on to play Frankenstein's monster for Unuiversal in two years, does a Lenny impression from "Of Mice and Men", it seems.<br /><br />*1/2 (of Four)


encoding, decoding 둘다 가능하다.

In [57]:
print('Tokenized sample question: {}'.format(tokenizer.encode(train_df['review'][20])))

Tokenized sample question: [1590, 4162, 132, 7107, 1892, 2983, 578, 76, 12, 4632, 3422, 7, 160, 175, 372, 2, 5, 39, 8051, 8, 84, 2652, 497, 39, 8051, 8, 1374, 5, 3461, 2012, 48, 5, 2263, 21, 4, 2992, 127, 4729, 711, 3, 1391, 8044, 3557, 1277, 8102, 2154, 5681, 9, 42, 15, 372, 2, 3773, 4, 3502, 2308, 467, 4890, 1503, 11, 3347, 1419, 8127, 29, 5539, 98, 6099, 58, 94, 4, 1388, 4230, 8057, 213, 3, 1966, 2, 1, 6700, 8044, 9, 7069, 716, 8057, 6600, 2, 4102, 36, 78, 6, 4, 1865, 40, 5, 3502, 1043, 1645, 8044, 1000, 1813, 23, 1, 105, 1128, 3, 156, 15, 85, 33, 23, 8102, 2154, 5681, 5, 6099, 8051, 8, 7271, 1055, 2, 534, 22, 1, 3046, 5214, 810, 634, 8120, 2, 14, 71, 34, 436, 3311, 5447, 783, 3, 6099, 2, 46, 71, 193, 25, 7, 428, 2274, 2260, 6487, 8051, 8, 2149, 23, 1138, 4117, 6023, 163, 11, 148, 735, 2, 164, 4, 5277, 921, 3395, 1262, 37, 639, 1349, 349, 5, 2460, 328, 15, 5349, 8127, 24, 10, 16, 10, 17, 8054, 8061, 8059, 8062, 29, 6, 6607, 8126, 8053]


In [58]:
# train_df에 존재하는 문장 중 일부를 발췌
sample_string = "It's mind-blowing to me that this film was even made."

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 : [137, 8051, 8, 910, 8057, 2169, 36, 7, 103, 13, 14, 32, 18, 79, 681, 8058]
기존 문장 : It's mind-blowing to me that this film was even made.


In [59]:
print('단어 집합의 크기(Vocab size) :', tokenizer.vocab_size)

단어 집합의 크기(Vocab size) : 8268


In [60]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

137 ----> It
8051 ----> '
8 ----> s 
910 ----> mind
8057 ----> -
2169 ----> blow
36 ----> ing 
7 ----> to 
103 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
79 ----> even 
681 ----> made
8058 ----> .


even이라는 단어에 뜬금없는 xyz를 붙여서 evenxyz를 넣어서 문장을 만들어도 잘 tokenize하는지 확인한다.

In [61]:
# 앞서 실습한 문장에 even 뒤에 임의로 xyz 추가
sample_string = "It's mind-blowing to me that this film was evenxyz made."

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 : [137, 8051, 8, 910, 8057, 2169, 36, 7, 103, 13, 14, 32, 18, 7974, 8132, 8133, 997, 681, 8058]
기존 문장 : It's mind-blowing to me that this film was evenxyz made.


In [62]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

137 ----> It
8051 ----> '
8 ----> s 
910 ----> mind
8057 ----> -
2169 ----> blow
36 ----> ing 
7 ----> to 
103 ----> me 
13 ----> that 
14 ----> this 
32 ----> film 
18 ----> was 
7974 ----> even
8132 ----> x
8133 ----> y
997 ----> z 
681 ----> made
8058 ----> .


evenxyz에서 even을 분레해내고, x, y, z는 처음보는 거여서 따로 분리되어 있다.

In [63]:
import pandas as pd
import urllib.request
import tensorflow_datasets as tfds

In [64]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')

In [65]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [66]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [67]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(train_data.document, target_vocab_size=2**13)

In [68]:
print(tokenizer.subwords[:100])

['. ', '..', '영화', '이_', '...', '의_', '는_', '도_', '다', ', ', '을_', '고_', '은_', '가_', '에_', '.. ', '한_', '너무_', '정말_', '를_', '고', '게_', '영화_', '지', '... ', '진짜_', '이', '다_', '요', '만_', '? ', '과_', '나', '가', '서_', '지_', '로_', '으로_', '아', '어', '....', '음', '한', '수_', '와_', '도', '네', '그냥_', '나_', '더_', '왜_', '이런_', '면_', '기', '하고_', '보고_', '하는_', '서', '좀_', '리', '자', '스', '안', '! ', '에서_', '영화를_', '미', 'ㅋㅋ', '네요', '시', '주', '라', '는', '오', '없는_', '에', '해', '사', '!!', '영화는_', '마', '잘_', '수', '영화가_', '만', '본_', '로', '그_', '지만_', '대', '은', '비', '의', '일', '개', '있는_', '없다', '함', '구', '하']


In [69]:
print(train_data['document'][20])

나름 심오한 뜻도 있는 듯. 그냥 학생이 선생과 놀아나는 영화는 절대 아님


In [70]:
print('Tokenized sample question: {}'.format(tokenizer.encode(train_data['document'][20])))

Tokenized sample question: [669, 4700, 17, 1749, 8, 96, 131, 1, 48, 2239, 4, 7466, 32, 1274, 2655, 7, 80, 749, 1254]


In [71]:
sample_string = train_data['document'][21]

# 인코딩한 결과를 tokenized_string에 저장
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 : {}'.format(tokenized_string))

# 이를 다시 디코딩
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장 : {}'.format(original_string))

정수 인코딩 후의 문장 : [570, 892, 36, 584, 159, 7091, 201]
기존 문장 : 보면서 웃지 않는 건 불가능하다


In [72]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

570 ----> 보면서 
892 ----> 웃
36 ----> 지 
584 ----> 않는 
159 ----> 건 
7091 ----> 불가능
201 ----> 하다


## Huggingface Tokenizer

자주 등장하는 subword들을 하나의 token으로 취급하는 다양한 subword tokenizer를 제공한다.

In [73]:
pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.5 MB/s eta 0:00:00


## BERT Word Piece Tokenizer

In [74]:
import pandas as pd
import urllib.request
from tokenizers import BertWordPieceTokenizer

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f44c7fe5cc0>)

In [75]:
naver_df = pd.read_table('ratings.txt')
naver_df = naver_df.dropna(how='any')
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [76]:
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)

In [77]:
data_file = 'naver_review.txt'
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

tokenizer.train(files=data_file, vocab_size=vocab_size, limit_alphabet=limit_alphabet, min_frequency=min_frequency)

In [78]:
# vocab 저장
tokenizer.save_model('./')

['./vocab.txt']

In [79]:
# vocab 로드
df = pd.read_fwf('vocab.txt', header=None)
df

,0
0,[PAD]
1,[UNK]
2,[CLS]
3,[SEP]
4,[MASK]
...,...
29995,말라는
29996,말밖에는
29997,맘을
29998,맛도


In [80]:
encoded = tokenizer.encode('아 배고픈데 짜장면먹고싶다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['아', '배고', '##픈', '##데', '짜장면', '##먹고', '##싶다']
정수 인코딩 : [2111, 20628, 3692, 3291, 24680, 7871, 7379]
디코딩 : 아 배고픈데 짜장면먹고싶다


In [81]:
encoded = tokenizer.encode('커피 한잔의 여유를 즐기다')
print('토큰화 결과 :',encoded.tokens)
print('정수 인코딩 :',encoded.ids)
print('디코딩 :',tokenizer.decode(encoded.ids))

토큰화 결과 : ['커피', '한잔', '##의', '여유', '##를', '즐기', '##다']
정수 인코딩 : [12825, 25646, 3282, 12696, 3396, 10784, 3256]
디코딩 : 커피 한잔의 여유를 즐기다


이외에도 여러가지 다른 tokenizer들고 사용할 수 있다.

In [82]:
from tokenizers import ByteLevelBPETokenizer, CharBPETokenizer, SentencePieceBPETokenizer